In [44]:
import pandas as pd
import numpy as np
from tqdm import tqdm

# Load data

In [2]:
def concat_df(prefix=None):
    path = Path('.')
    files = list(path.glob(f'/{prefix}_*.csv'))

    dfs = [pd.read_csv(file) for file in files]
    total_df = pd.concat(dfs, ignore_index=True)

    return total_df

In [3]:
# resumes_df = concat_df('resumes')
# vacancies_df = concat_df('vacancies')

In [4]:
resumes_df = pd.read_csv(r'Total_dataset/total_resumes.csv')
vacancies_df = pd.read_csv(r'Total_dataset/total_vacancies.csv')
matches_df = pd.read_csv(r'Total_dataset/matches.csv')

# Preprocessing

## matches_df

In [5]:
matches_df['resume_ids'] = matches_df['resume_ids'].apply(lambda x: eval(x))

In [6]:
matches_df = matches_df.explode('resume_ids')

In [7]:
matches_df = matches_df.rename(columns={'resume_ids': 'resume_id'})

In [8]:
matches_df.duplicated().sum()

np.int64(13935)

In [9]:
matches_df = matches_df.drop_duplicates()

In [10]:
matches_df.loc[:,'resume_id'] = matches_df['resume_id'].astype(int)

In [11]:
matches_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 25525 entries, 0 to 3945
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   vacancy_id  25525 non-null  int64 
 1   resume_id   25525 non-null  object
dtypes: int64(1), object(1)
memory usage: 598.2+ KB


In [12]:
matches_df.head()

,vacancy_id,resume_id
0,126167948,6969174
0,126167948,9100077
0,126167948,32644957
0,126167948,27220466
0,126167948,7532708


## resumes_df

In [13]:
resumes_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25229 entries, 0 to 25228
Data columns (total 19 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              25229 non-null  int64  
 1   title                           25096 non-null  object 
 2   url                             25229 non-null  object 
 3   specialization                  25091 non-null  object 
 4   last_company                    24594 non-null  object 
 5   last_position                   24603 non-null  object 
 6   last_experience_description     24589 non-null  object 
 7   last_company_experience_period  24604 non-null  object 
 8   skills                          17028 non-null  object 
 9   education                       24760 non-null  object 
 10  courses                         10607 non-null  object 
 11  salary                          14481 non-null  object 
 12  age                             

In [14]:
resumes_df = resumes_df.sort_values('id')

Проверим наличие дубликатов по всем столбцам и только по id

In [15]:
resumes_df.duplicated().sum().item()

3

In [16]:
resumes_df['id'].duplicated().sum().item()

4318

Т.к. резюме скачиваются по разным профессиям и в разное время, могут скачиваться одни и те же резюме. Оставим только последнюю скаченную версию.

In [17]:
resumes_df['rank'] = resumes_df.groupby('id')['parsed_date'].rank(ascending=False)
resumes_df = resumes_df[resumes_df['rank'] == 1]

Оставим только необходимые для обучения колонки.

In [18]:
old_resume_col = ['id', 'title', 'specialization', 'last_position', 'last_experience_description', 'last_company_experience_period', 
                  'skills', 'education', 'courses', 'salary', 'age', 'total_experience', 'experience_months', 'location', 'gender', 'applicant_status']
new_resume_col = ['resume_' + col for col in old_resume_col]
new_resume_col_dict = {old_col: new_col for old_col, new_col in zip(old_resume_col, new_resume_col)}
resumes_df = resumes_df[old_resume_col].rename(columns=new_resume_col_dict)

In [19]:
resumes_df.head()

,resume_id,resume_title,resume_specialization,resume_last_position,resume_last_experience_description,resume_last_company_experience_period,resume_skills,resume_education,resume_courses,resume_salary,resume_age,resume_total_experience,resume_experience_months,resume_location,resume_gender,resume_applicant_status
15067,201,Finance director,"['Финансовый аналитик, инвестиционный аналитик...",Production Financial controller,•\tOverall financial control of multipurpose p...,4 года 5 месяцев,"['Budgeting', 'IFRS', 'SAP', 'Risk management'...","['London Guildhall Metropolitan University', '...","['Japanese Educational Centre, Master', 'Mosco...",400 000 ₽ на руки,NaN,27 лет 10 месяцев,334.0,Москва,Мужчина,Активно ищет работу
22849,349,Специалист по Информационным технологиям,['Специалист технической поддержки'],Автомеханик,"Ремонт VW, BMW, Ford, Mitsubishi, Opel, Toyota...",24 года 5 месяцев,NaN,['Московский государственный институт радио...,['Школа юного менеджера при РЭА имени Г.В. Пле...,700 $ на руки,48.0,26 лет 2 месяца,314.0,Москва,Мужчина,NaN
12826,738,Менеджер,"['Менеджер по продажам, менеджер по работе с к...",индивидуальный предприниматель,"Персональные компьютеры, сотовая и радиосвязь,...",29 лет 5 месяцев,NaN,"['Мос. Электротехнический Институт Связи', 'Мо...",NaN,1 000 $ на руки,78.0,55 лет 10 месяцев,670.0,Москва,Мужчина,NaN
12827,940,Технический директор,['Начальник производства'],Head of production department,Руководсво полным циклом производства:выбор пр...,24 года 11 месяцев,NaN,['Московский Физико-технический институт'],NaN,1 500 $ на руки,56.0,30 лет 6 месяцев,366.0,Москва,Мужчина,NaN
21517,1079,Программист,"['Программист, разработчик']",Ведущий разработчик,Создал микросервисы на Python для загрузки зад...,3 года 7 месяцев,"['ORACLE', 'MS SQL', 'MySQL', 'PostgreSQL', 'J...",['Московский государственный инженерно-физичес...,"['OTUS, NLP / Natural Language Processing', 'G...",400 000 ₽ на руки,53.0,31 год 1 месяц,373.0,Москва,Мужчина,Активно ищет работу


## vacancies_df

In [20]:
vacancies_df.duplicated().sum().item()

2

In [21]:
vacancies_df['id'].duplicated().sum().item()

536

In [22]:
vacancies_df['rank'] = vacancies_df.groupby('id')['parsed_date'].rank(ascending=False)
vacancies_df = vacancies_df[vacancies_df['rank'] == 1]

In [23]:
old_vacancy_col = ['id', 'name', 'area', 'experience', 'employment', 'schedule', 'salary_from', 'salary_to', 'salary_currency', 'salary_gross', 'description']
new_vacancy_col = ['vacancy_' + col for col in old_vacancy_col]
new_vacancy_col_dict = {old_col: new_col for old_col, new_col in zip(old_vacancy_col, new_vacancy_col)}
vacancies_df = vacancies_df[old_vacancy_col].rename(columns=new_vacancy_col_dict)

In [24]:
vacancies_df.head()

,vacancy_id,vacancy_name,vacancy_area,vacancy_experience,vacancy_employment,vacancy_schedule,vacancy_salary_from,vacancy_salary_to,vacancy_salary_currency,vacancy_salary_gross,vacancy_description
0,126167948,Разработчик SAP ABAP,Москва,Более 6 лет,Полная занятость,Удаленная работа,300000.0,NaN,RUR,False,"Привет!.redev — технологическая компания, созд..."
1,126211419,Старший разработчик ABAP,Москва,От 3 до 6 лет,Полная занятость,Удаленная работа,NaN,NaN,NaN,NaN,Лента — федеральный продуктовый ритейлер Росси...
2,123101841,Старший разработчик SAP ABAP (Clean Core) (Чер...,Москва,От 3 до 6 лет,Полная занятость,Полный день,NaN,NaN,NaN,NaN,Наша вакансия подразумевает релокацию в Черног...
3,125903052,ABAP-разработчик,Москва,Более 6 лет,Полная занятость,Удаленная работа,NaN,NaN,NaN,NaN,ТерраЛинк - многопрофильный интегратор с 30-ле...
4,121722062,Консультант-эксперт SAP BW/BI,Москва,От 3 до 6 лет,Полная занятость,Удаленная работа,NaN,NaN,NaN,NaN,Компания СИБИНТЕК-СОФТ приглашает на вакансию ...


In [25]:
vacancies_df.isna().sum() / len(vacancies_df)

vacancy_id                 0.000000
vacancy_name               0.000000
vacancy_area               0.000000
vacancy_experience         0.000000
vacancy_employment         0.000000
vacancy_schedule           0.000000
vacancy_salary_from        0.768554
vacancy_salary_to          0.813435
vacancy_salary_currency    0.709006
vacancy_salary_gross       0.709006
vacancy_description        0.000000
dtype: float64

In [26]:
vacancies_df.shape

(3409, 11)

# Merging

In [27]:
total_df = vacancies_df.merge(matches_df, how='left')

In [28]:
total_df = total_df.merge(resumes_df, how='left')

In [46]:
total_df['target'] = 1

Добавляем неподходящие к вакансии резюме для обучения

In [47]:
target_zero_list = []
for vacancy in tqdm(vacancies_df['vacancy_id'].unique()):
    resume_match_list = matches_df.loc[matches_df['vacancy_id'] == vacancy, 'resume_id'].tolist()
    df_target_zero = vacancies_df[vacancies_df['vacancy_id'] == vacancy].merge(resumes_df[~resumes_df['resume_id'].isin(resume_match_list)].sample(10), how='cross')
    target_zero_list.append(df_target_zero)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3409/3409 [00:14<00:00, 233.66it/s]


In [49]:
df_target_zero = pd.concat(target_zero_list, axis=0)

In [52]:
df_target_zero['target'] = 0

Соединяем обе части в единый датафрейм

In [53]:
total_df = pd.concat([total_df, df_target_zero], axis=0)

In [55]:
total_df.to_csv(r'Total_dataset/total_df.csv', index=False)